# Chapter 5: Functions?!
Boy howdy, do I love functions. Eat your heart out Java!!!

## 30: Know that Function Arguments can be Mutated
Python aint got them fancy pointer doodads, but that doesn't mean you're always safe passing things around.

In [36]:
def mutant_attack(items):
    items.append(4) # mutate what the reference points at

x = [1,2,3]
mutant_attack(x)
x

[1, 2, 3, 4]

In [37]:
def replace_em(items):
    items = [9,8,7] # trying to mutate the reference itself

y = [1,2,3]
replace_em(y)
y

[1, 2, 3]

In [38]:
z = x
mutant_attack(z)
x

[1, 2, 3, 4, 4]

In [19]:
class MyClass:
    def __init__(self,value):
        self.value = value

x = MyClass(10)


def my_mutator(obj):
    obj.value = 20

my_mutator(x)

x.value

20

## 31: Return Dedicated Result Objects Instead of Requiring Function Callers to Unpack More Than Three Variables
Lets take a look at a meaty function:

In [24]:
from statistics import median as built_in_median

def gimme_stats(nums):
    minimum = min(nums)
    maximum = max(nums)
    count = len(nums)
    average = sum(nums) / count
    median = built_in_median(nums)
    return minimum, maximum, average, median, count

minimum, maximum, average, median, count = gimme_stats([1,2,3,4,5]) # lame
gimme_stats([1,2,3,4,5]) 

(1, 5, 3.0, 3, 5)

This sucks. What happens if you got things in the wrong order, or had the wrong number of unpacking vars? BOO

Dataclasses to the rescue!
Eric shameless brag moment, after reading the title of this item I thought "oh yeah definitely use a dataclass"

(almost as if someone has read the previous edition of this book 🤔)

In [26]:
@dataclass
class Stats:
    minimum: float
    maximum: float
    average: float
    median: float
    count: int

def gimme_stats(nums):
    return Stats(
        minimum=min(nums),
        maximum=max(nums),
        count=len(nums),
        average=sum(nums) / count,
        median=built_in_median(nums),
    )

those_stats = gimme_stats([1,2,3,4,5,6,7,8,9])
those_stats

Stats(minimum=1, maximum=9, average=9.0, median=5, count=9)

This is easier to work with, and lets you fetch things by their names too like `those_stats.average` etc

## 32: Prefer Raising Exceptions to Returning `None`
Whoops, Eric did this in a few spots in `swordfish`! Loser.

#### For the Java-Afflicted
Returning `None` is basically as egregious as returning `null`. There's no reason to do it in modern Java.

In [31]:
# A seemingly reasonable case
def careful_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None

x, y = 1, 0
result = careful_divide(x, y)
if result is None:
    print("dang it, we divided by zero!!!")

dang it, we divided by zero!!!


In [32]:
# Where it gets a bit messy:
x, y = 0, 5
result = careful_divide(x, y)
if not result:
    print("Uhh.... wait this wasn't a None")

Uhh.... wait this wasn't a None


In [35]:
# You could do it like this, as the book explains
def careful_divide_bad_no(a, b):
    try:
        return True, a / b
    except ZeroDivisionError:
        return False, None

# But lets be real, this is bad API design. We foist extra work on callers,
# and there's no guarantee they'll heed it anyway..
# they might even think the first value is the result. It's bad! BOO

_, result = careful_divide_bad_no(x, y)
if not result:
    print("Invalid inputs")

Invalid inputs


In [36]:
# In Python, exceptions aren't always _exceptional_

def careful_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        raise ValueError("Invalid inputs")

x, y = 5, 2
try:
    result = careful_divide(x, y)
except ValueError:
    print("Invalid inputs")
else:
    print(f"Result is {result:.1f}")


Result is 2.5


This is kind of a trite example, because honestly we could always just _not_ catch/handle the `ZeroDivisionError` in the function and put that onus on the caller.

A better way to think about this would be with an API and a Database:

In [38]:
class MyDbClient:
    """A really bad DB client."""
    def connect(self):
        raise ValueError("I REFUSE to connect, you CANNOT make me!")

my_trustworthy_db_client = MyDbClient()

db_connection = my_trustworthy_db_client.connect()

ValueError: I REFUSE to connect, you CANNOT make me!

In [50]:
class MyHttpException(Exception):
    """An error your webserver knows how to handle."""
    def __init__(self, message, error_code):
        super().__init__(message)
        self.error_code = error_code

def my_get_route():
    """Gets the item from the database.

    Raises:
        MyHttpException: When the database is being goofy.
    """
    try:
        db_connection = my_trustworthy_db_client.connect()
    except ValueError as e:
        raise MyHttpException("Something goofed.", 500) from e

# my_get_route()

PAUSE: what's `from e`?

In [51]:
def my_top_handler():
    try:
        return my_get_route()
    except MyHttpException as e:
        return e.error_code

my_top_handler()

500

# 33: Know How Closures Interact with Variable Scope and `nonlocal`

In [57]:
def sort_notifications(values, group):
    def sort_key(x): # define our sort key as a closure function
        if x in group:
            return (0, x) # rely on built-in sequence comparison behavior
        return (1, x)

    values.sort(key=sort_key)

# imagine these are notification IDs
notifications = [8, 3, 1, 2, 5, 4, 7, 6]
high_priority = {2, 3, 5, 7}
sort_notifications(notifications, high_priority)
notifications

# for each notification send to front end blah blah

[2, 3, 5, 7, 1, 4, 6, 8]

Oops, new requirements: the FE team would like to _know_ if any of the high priority notifications were seen or not.

In [1]:
def sort_notifications2(values, group):
    found = False
    
    def sort_key(x):
        if x in group:
            found = True # we found one!!!
            return (0, x)
        return (1, x)

    values.sort(key=sort_key)
    return found

notifications = [8, 3, 1, 2, 5, 4, 7, 6]
high_priority = {2, 3, 5, 7}
found = sort_notifications2(notifications, high_priority)
print(f"Found: {found}; order: {notifications}")

Found: False; order: [2, 3, 5, 7, 1, 4, 6, 8]


![spongebob.png](spongebob.png)

### Okay, so what the hell?
The `found = True` is treated as _variable assignment_ by the Python interpreter. Essentially, it traversed the scope and immediately found `found = True` which is a valid variable assignment, so it stopped here, and determined this expression was creating a new variable `found` in the local scope. It never traversed back up to see the `found = False`

This is intentional behavior, to keep the global module scope clean.

In [3]:
def sort_notifications2(values, group):
    found = False
    
    def sort_key(x):
        nonlocal found  # give Python a lil hint...
        if x in group:
            found = True
            return (0, x)
        return (1, x)

    values.sort(key=sort_key)
    return found

notifications = [8, 3, 1, 2, 5, 4, 7, 6]
high_priority = {2, 3, 5, 7}
found = sort_notifications2(notifications, high_priority)
print(f"Found: {found}; order: {notifications}")

Found: True; order: [2, 3, 5, 7, 1, 4, 6, 8]


And that made it work! BUT, we should use `nonlocal` (and its counterpart `global`) sparringly, as these are almost like GOTOs for variables. Spooky shit bro.

# 34: Reduce Visual Noise with Variable Positional Arguments
Remember the unpacking syntax? It's back again.

In [34]:
def raise_p0(message, values):
    if not values:
        print(message)
    else:
        values_str = ", ".join(str(x) for x in values)
        print(f"{message}: {values_str}")

raise_p0("Video Center is showing me scary numbers", [1, 2])
raise_p0("I stubbed my toe and now I can't reach my computer to publish my photo", [])

Video Center is showing me scary numbers: 1, 2
I stubbed my toe and now I can't reach my computer to publish my photo


Sweet, we STREAMLINED the process for customers to raise issues and hold us accountable! But, well... this function could be better:

`raise_p0("I only want to give one argument!", [])`

In [8]:
def raise_p0(message, *values):
    if not values:
        print(message)
    else:
        values_str = ", ".join(str(x) for x in values)
        print(f"{message}: {values_str}")

raise_p0("I don't like teal", 1, 2, "please use red")
raise_p0("Test please ignore")

I don't like teal: 1, 2, please use red
Test please ignore


In [12]:
# You can use this WITH unpacking! Wow!
litany_of_complaints = ["its bad", "its expensive", "jimmy giving me weird looks"]

raise_p0("I'm sick and tired, and wont take it anymore!", *litany_of_complaints)

I'm sick and tired, and wont take it anymore!: its bad, its expensive, jimmy giving me weird looks


#### Warning: Remember Generators?
Consider the following

In [11]:
def lots_of_numbers():
    for i in range(1_000_000_000_000):
        yield i

it = lots_of_numbers()

# Probably don't run this example <_<
# raise_p0("hellos, I am friendli not-hacker", *it)

Another annoyance: what if you refactor to add a positional arg?

In [15]:
def raise_support(priority, message, *values):
    if not values:
        print(message)
    else:
        values_str = ", ".join(str(x) for x in values)
        print(f"P{priority} - {message}: {values_str}")

raise_support(1, "help", "something is broken!") # fine
raise_support("I'm sick and tired, and wont take it anymore!", *litany_of_complaints) # oops
# because of the *arg, the function will happily accept calls with variable inputs
# Python won't help you catch this bug

P1 - help: something is broken!
PI'm sick and tired, and wont take it anymore! - its bad: its expensive, jimmy giving me weird looks


Type annotations can protect you a bit here:
```python
def raise_support(priority: int, message: str, *values: Any):```

# 35: Provide Optional Behavior with Keyword Arguments
This is the one I miss the most when I'm working in Java.

In [19]:
def foo_much(foo, bar, baz=1):
    print(f"FOO {foo}, BAR {bar}, BAZ {baz}")

foo_much(1,2)
foo_much(bar=2, foo=1)

FOO 1, BAR 2, BAZ 1
FOO 1, BAR 2, BAZ 1


In [22]:
# Bad
# foo_much(foo=1, bar=2, foo=2)
# foo_much(bar=2, 1)

In [25]:
foobar = {"foo": 22, "bar": 99}
foo_much(**foobar)
foo_much(**foobar, baz=9000)

FOO 22, BAR 99, BAZ 1
FOO 22, BAR 99, BAZ 9000


In [26]:
baz = {"baz": 8888}
foo_much(**baz, **foobar) # note, no overlapping keys

FOO 22, BAR 99, BAZ 8888


In [29]:
# Similar to the variable positional args, we can do this for keyword args (KWARGS) too:
def generic_much(**kwargs):
    for key, value in kwargs.items():
        print(f"{key} {value}".upper())

generic_much(foo=1, bar=2, baz=3)

FOO 1
BAR 2
BAZ 3


# 36: Use `None` and Docstrings to Specify Dynamic Default Arguments
Would you look at that, we got new requirements for our ACS python SDK! They want timestamps?!

In [40]:
from time import sleep
from datetime import datetime

def raise_support(priority, message, *values, when=datetime.now()):
    if not values:
        print(f"{when}: P{priority} - {message}")
    else:
        values_str = ", ".join(str(x) for x in values)
        print(f"{when}: P{priority} - {message}: {values_str}")

raise_support(1, "help", "something is broken!")
sleep(0.2)
raise_support(5, "I just wanted to talk")

2025-04-16 17:07:48.891218: P1 - help: something is broken!
2025-04-16 17:07:48.891218: P5 - I just wanted to talk


Wait a minute... those are the same timestamps
![huh.jpg](huh.jpg)

Yeah! When Python first encounters `raise_support` while evaluating source code, it executes and saves the result of that first `datetime.now()`.

In [42]:
# The conventional way

def raise_support(priority, message, *values, when=None):
    """Raise the support team

    Args:
        ...
        when: datetime of when the message occured, defaults to now.
    """
    if when is None:
        when = datetime.now()
    if not values:
        print(f"{when}: P{priority} - {message}")
    else:
        values_str = ", ".join(str(x) for x in values)
        print(f"{when}: P{priority} - {message}: {values_str}")

raise_support(1, "help", "something is broken!")
sleep(0.2)
raise_support(5, "I just wanted to talk")

2025-04-16 17:16:46.951552: P1 - help: something is broken!
2025-04-16 17:16:47.156594: P5 - I just wanted to talk


A BRIEF ASIDE: datetime.now()

Likely your LLM will tell you to use `datetime.utcnow()` if you want `now` with timezone info. This is the old and wrong way. Instead, do thus:

```python
from datetime import UTC

datetime.now(UTC)
```


# 37: Enforce Clarity with Keyword-Only and Positional-Only Arguments

In [46]:
def get_resizer_support(priority, message_for_jimmy, *, direct_to_jimmy=True):
    if direct_to_jimmy:
        print("JIMMY PLZ")
    else:
        print(f"P{priority} - {message}")

get_resizer_support(0, "RESIZE GIF", True)

TypeError: get_resizer_support() takes 2 positional arguments but 3 were given

In [47]:
get_resizer_support(0, message_for_jimmy="RESIZE GIF", direct_to_jimmy=True)

JIMMY PLZ


Uh oh, this wont work, what if we finally figure out how to support resizer _without_ bothering Jimmy? We gotta make sure people don't use that named `message_for_jimmy` bit, otherwise we'll break them when we update the function!

In [50]:
def get_resizer_support(priority, message_for_jimmy, /, *, direct_to_jimmy=True):
    if direct_to_jimmy:
        print("JIMMY PLZ")
    else:
        print(f"P{priority} - {message}")

get_resizer_support(0, message_for_jimmy="RESIZE GIF")

TypeError: get_resizer_support() got some positional-only arguments passed as keyword arguments: 'message_for_jimmy'

In [51]:
get_resizer_support(0, "RESIZE GIF")

JIMMY PLZ


In [54]:
def foo(x, /):
    print(x)

foo(1)

1


# 38: Define Function Decorators with `functools.wraps`

In [66]:
# an example tracing decorator
def trace(func):
    def wrapper(*args, **kwargs):
        args_repr = repr(args)
        kwargs_repr = repr(kwargs)
        result = func(*args, **kwargs)
        print(f"{func.__name__}"
              f"({args_repr}, {kwargs_repr}) "
              f"-> {result!r}")
        return result

    return wrapper

In [67]:
# lets trace a fibbo

@trace
def fibonacci(n):
    """Return the n-th Fibonacci number"""
    if n in (0, 1):
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

fibonacci(3)

fibonacci((1,), {}) -> 1
fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((3,), {}) -> 2


2

In [68]:
# typical decorator problem, the function loses its name!
help(fibonacci)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



In [69]:
# this breaks serialization too (which matters for distributed runners, or machine-learning
import pickle

pickle.dumps(fibonacci)

AttributeError: Can't get local object 'trace.<locals>.wrapper'

In [70]:
# Easy as heck to solve though, just use functools.wraps!!!!!!
from functools import wraps

def trace(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        args_repr = repr(args)
        kwargs_repr = repr(kwargs)
        result = func(*args, **kwargs)
        print(f"{func.__name__}"
              f"({args_repr}, {kwargs_repr}) "
              f"-> {result!r}")
        return result

    return wrapper

@trace
def fibonacci(n):
    """Return the n-th Fibonacci number"""
    if n in (0, 1):
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

help(fibonacci)

Help on function fibonacci in module __main__:

fibonacci(n)
    Return the n-th Fibonacci number



In [71]:
# and now pickling works too!
print(pickle.dumps(fibonacci))

b'\x80\x04\x95\x1a\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\tfibonacci\x94\x93\x94.'


# 39: Prefer `functools.partial` over `lambda` Expressions for Glue Functions
Time for some function currying.

In [ ]:
# Actually, lets go look at swordfish